In [1]:
from quantum_field import *
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
import pandas as pd
import numpy as np
import datashader as ds
from colorcet import fire, kbc, bmw, gray
template="plotly_dark"

In [2]:
# Construct the slits
wallMass = 10
yWall = 10
slitAperture = 100
leftEdge = Object(-slitAperture / 2, yWall, wallMass)
leftWall = [Object(-100 - slitAperture / 2, yWall, wallMass), leftEdge]
rightEdge = Object(slitAperture / 2, yWall, wallMass)
rightWall = [Object(100 + slitAperture / 2, yWall, wallMass), rightEdge]
edges = [leftEdge, rightEdge]
walls = [leftWall, rightWall]

# Uncomment for one slit
# edges = [Object(i+slitAperture/2, yWall, wallMass) for i in range(10)]
edges = [rightEdge]
walls = [edges]


# Create the particles
v_y_particles = 300
particle_y0_step = 5
particle_x0_fraction = 10

particles = [
    Particle(x_t0=i/particle_x0_fraction, y_t0=j, v_x=0, v_y=v_y_particles, mass=1) for j in range(-100, 0, particle_y0_step)
    for i in range(0, (slitAperture//2)*particle_x0_fraction-1, 1)
]


# Define parameters
dt = 1
cos_sin = "none"
power = 1
r_modifier = 1
G=100
c=v_y_particles
gravityCutoff = 1000
yGrid = True

# Punch it
for t in range(100):
    for particle in particles:
        if particle.y > yWall*2:
            dt = 1
        if particle.v_y > 0:
            updateParticle(
                particle,
                edges,
                G=G,
                dt=dt,
                r_modifier=r_modifier,
                c=c,
                power=power,
                gravityCutoff=gravityCutoff,
                cos_sin=cos_sin,
                yGrid=yGrid,
            )

In [3]:
grid = pd.DataFrame(np.vstack((np.hstack([particle.x_trail for particle in particles]), np.hstack([particle.y_trail for particle in particles])))).T
grid.rename(columns={0: "x", 1: "y"}, inplace=True)
grid["x"] = np.round(grid.x, 0)
grid.pivot_table(index="y", columns="x", aggfunc=len, fill_value=0).to_numpy()


array([[12, 18, 22, ...,  0,  0,  0],
       [ 5, 10, 10, ...,  0,  0,  0],
       [ 5, 10, 10, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  1,  0,  0],
       [ 0,  0,  0, ...,  0,  1,  0],
       [ 0,  0,  0, ...,  0,  0,  1]])

# Graphs

In [ ]:
fig = go.Figure(
    data=[
        go.Scattergl(
            x=particle.x_trail,
            y=particle.y_trail,
            mode="lines",
            marker_size=5,
            marker_color="crimson",
            line_width=1,
        )
        for particle in particles
    ]
    + [
        go.Scattergl(
            x=[particle.x for particle in particles],
            y=[particle.y for particle in particles],
            mode="markers",
            marker_color="red",
            marker_size=2,
            line_width=1,
        )
    ]
    + [
        go.Scattergl(
            x=[edge.x for edge in wall],
            y=[edge.y for edge in wall],
            mode="markers+lines",
            marker_color="darkgray",
        )
        for wall in walls
    ],
    layout=dict(width=1600, height=800),
)
fig.update_layout(template=template)
fig.write_html(
    f"plotly_graphs/_test_2d_{len(particles)}p_ygrid{yGrid}_apert{slitAperture}_fract{particle_x0_fraction}_{cos_sin}{power}_rmod{r_modifier}_G{G}_c{c}_y0_step{particle_y0_step}_gravCut{gravityCutoff}.html"
)
# fig.show()
